RNN Classifier

Tutorial:
https://morvanzhou.github.io/tutorials/machine-learning/keras/2-4-RNN-classifier/

In [2]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import SimpleRNN, Activation, Dense
from keras.optimizers import Adam

/home/anthony/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
TIME_STEPS = 28     # same as the height of the image
INPUT_SIZE = 28     # same as the width of the image
BATCH_SIZE = 50
BATCH_INDEX = 0
OUTPUT_SIZE = 10
CELL_SIZE = 50
LR = 0.001

In [4]:
# download the mnist to the path '~/.keras/datasets/' if it is the first time to be called
# X shape (60,000 28x28), y shape (10,000, )
(X_train, y_train), (X_test, y_test) = mnist.load_data()

# data pre-processing
X_train = X_train.reshape(-1, 28, 28) / 255.      # normalize
X_test = X_test.reshape(-1, 28, 28) / 255.        # normalize
y_train = np_utils.to_categorical(y_train, num_classes=10)
y_test = np_utils.to_categorical(y_test, num_classes=10)

In [5]:
X_train.shape

(60000, 28, 28)

In [6]:
model = Sequential()

In [7]:
model.add(SimpleRNN(
    # for batch_input_shape, if using tensorflow as the backend, we have to put None for the batch_size.
    # Otherwise, model.evaluate() will get error.
    batch_input_shape=(None, TIME_STEPS, INPUT_SIZE),       
    output_dim=CELL_SIZE,
    unroll=True,
))

/home/anthony/.local/lib/python3.5/site-packages/ipykernel_launcher.py:6: UserWarning: Update your `SimpleRNN` call to the Keras 2 API: `SimpleRNN(unroll=True, batch_input_shape=(None, 28,..., units=50)`
  


In [8]:
# output layer
model.add(Dense(OUTPUT_SIZE))
model.add(Activation('softmax'))

In [9]:
# optimizer
adam = Adam(LR)
model.compile(optimizer=adam,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
for step in range(4001):
    # data shape = (batch_num, steps, inputs/outputs)
    X_batch = X_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :, :]
    Y_batch = y_train[BATCH_INDEX: BATCH_INDEX+BATCH_SIZE, :]
    cost = model.train_on_batch(X_batch, Y_batch)
    BATCH_INDEX += BATCH_SIZE
    BATCH_INDEX = 0 if BATCH_INDEX >= X_train.shape[0] else BATCH_INDEX

    if step % 500 == 0:
        cost, accuracy = model.evaluate(X_test, y_test, batch_size=y_test.shape[0], verbose=False)
        print('test cost: ', cost, 'test accuracy: ', accuracy)

test cost:  2.4057374000549316 test accuracy:  0.03909999877214432
test cost:  0.6074601411819458 test accuracy:  0.8180000185966492
test cost:  0.44737547636032104 test accuracy:  0.8644000291824341
test cost:  0.3490295112133026 test accuracy:  0.8980000019073486
test cost:  0.3177788555622101 test accuracy:  0.9052000045776367
test cost:  0.28951436281204224 test accuracy:  0.9154000282287598
test cost:  0.25214582681655884 test accuracy:  0.9269999861717224
test cost:  0.23782452940940857 test accuracy:  0.9269000291824341
test cost:  0.23364819586277008 test accuracy:  0.9311000108718872


这个感觉就是RNN做特征提取，将图像转换成一个50维的向量，然后用一个全连接层来做分类。并不是RNN当做分类器来用了。